### Initial Setup

In [2]:
import os, sys
import pprint as p
import pyspark.sql.functions as pysF
%matplotlib widget
%matplotlib inline

py_file_path = os.path.join(
    os.getcwd(),
    "..",
    ".."
)

sys.path.append(py_file_path)
from app.SparkTools import MyPySpark

MySpark = None

#ensure only one sc and spark instance is running
global MySpark
MySpark = MySpark or MyPySpark(
    master = 'local[3]', 
    logger_name = 'jupyter')

### Filter and Join Data

In [6]:
p.pprint(
    list(
        set(
            MySpark\
            .spark\
            .read\
            .parquet("/Processed/ElectricityDimDF")\
            .select("value_type")
            .toPandas()["value_type"])))

['Fuel consumption MMBtu',
 'Consumption for electricity generation',
 'Revenue from retail sales of electricity',
 'Consumption for electricity generation (Btu)',
 'Electric fuel consumption quantity',
 'Total consumption',
 'Electric fuel consumption MMBtu',
 'Number of customer accounts',
 'Average retail price of electricity',
 'Consumption for useful thermal output',
 'Average cost of fossil fuels for electricity generation',
 'Receipts of fossil fuels by electricity plants',
 'Average cost of fossil fuels for electricity generation (per Btu)',
 'Consumption for useful thermal output (Btu)',
 'Net generation',
 'Quality of fossil fuels in electricity generation',
 'Total consumption (Btu)',
 'Receipts of fossil fuels by electricity plants (Btu)',
 'MMBtu per unit',
 'Fuel consumption quantity',
 'Retail sales of electricity',
 'Fossil-fuel stocks for electricity generation']


In [15]:
electricity_dim_df = MySpark\
    .spark\
    .read\
    .parquet("/Processed/ElectricityDimDF")\
    .select("series_id", "state", "fuel_type", "engine_type", "units")\
    .filter(pysF.col("value_type") == 'Fuel consumption MMBtu')

electricity_fact_df = MySpark\
    .spark\
    .read\
    .parquet("/Processed/ElectricityFactDF")

electricity_df = electricity_fact_df.join(
    pysF.broadcast(electricity_dim_df),
    on = "series_id",
    how = "right"
)

In [20]:
electricity_df\
    .withColumn(
        "year", 
        pysF.year(pysF.col("date")))\
    .groupBy("year", "state")\
    .pivot("fuel_type")\
    .agg(pysF.sum("value"))\
    .show(20)

+----+-----+-----------+------------------------+--------------------+------------------------------+-------------------+-------------+-----------------+--------------------------+--------------------------+--------------------+---------------+----------+---------+------------------+--------------------+------------+---------------------+--------------------+----------------------------------+---------------+---------+-----------------+---------------------+--------------------+-----------------+--------------------+---------------+------------+-----------------+------------+---------+--------------------+--------------+------------------+----------+----------------+------------------+--------------------+-----------------------------------+----------------------+
|year|state|       null|agricultural by-products|           all fuels|biogenic municipal solid waste|    bituminous coal|black liquour|blast furnace gas|coal-derived synthetic gas|conventional hydroelectric| distillate fuel o